# Segmenting and Clustering Neighborhoods in Toronto
## Week 3

##### Importing all libraries

In [14]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 
from bs4 import BeautifulSoup
import urllib.request
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


##### Scraping the data from Wikipedia

In [15]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
table=soup.find('table', class_='wikitable sortable')
Postcode=[]
Borough=[]
Neighborhood=[]
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighborhood.append(cells[2].find(text=True))

      
df = pd.DataFrame(Postcode,columns=['Postcode'])
df['Borough']=Borough
df['Neighborhood']=Neighborhood

##### Cleaning up & arranging the data

In [16]:
df = df[df['Borough']!='Not assigned'] 
df.index = np.arange(0, len(df))

In [17]:
df['Neighborhood'] = df.groupby('Postcode')['Neighborhood'].transform(lambda x: "%s" % ','.join(x)).values
df = df.drop_duplicates().reset_index(drop=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned\n


In [18]:
df['Neighborhood'].replace("Not assigned\n", df['Borough'], inplace=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [19]:
df.shape

(103, 3)

##### Adding latitude and longitude information to the dataframe

In [20]:
df1= pd.read_csv('https://cocl.us/Geospatial_data')
df1 = df1.rename(index=str, columns={"Postal Code": "Postcode"})
df=df.merge(df1, how='left', on='Postcode')
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


##### Clustering and visualization

In [21]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes

import folium # plotting library

print('Folium installed')
print('Libraries imported.')address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: ...working... done

# All requested packages already installed.



WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


In [65]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\hello\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [66]:
import requests
# The latitude and longitude information of Toronto was manually extracted from Google 
latitude = 43.653963
longitude = -79.387207
radius = 500
limit = 30
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [69]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.7) # generate map centred around Toronto

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  

In [22]:
CLIENT_ID = '1T1BO50DCYM4O1WD0OUENVUYYRI2YAFL1HN5GO5I2CIIQ4KN' # your Foursquare ID
CLIENT_SECRET = 'PORDSZOO0E3DTAMF14SF14NOVCLIB1IGJBVL40LFLTYQIRB0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1T1BO50DCYM4O1WD0OUENVUYYRI2YAFL1HN5GO5I2CIIQ4KN
CLIENT_SECRET:PORDSZOO0E3DTAMF14SF14NOVCLIB1IGJBVL40LFLTYQIRB0


In [70]:
def getNearbyVenues(postcode, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for pc, names, lat, lng in zip(postcode, names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pc,
            names, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
toronto_venues = getNearbyVenues(postcode=df['Postcode'], names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
toronto_venues.head()

,Postcode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [74]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [76]:
#examine dataframe size
toronto_onehot.shape

(1331, 240)

In [81]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [82]:
toronto_grouped.shape

(100, 240)

In [86]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide
,King
,Richmond
----
              venue  freq
0        Steakhouse  0.10
1              Café  0.07
2       Pizza Place  0.07
3  Asian Restaurant  0.07
4             Hotel  0.07


----Agincourt----
                venue  freq
0              Lounge  0.25
1      Breakfast Spot  0.25
2  Chinese Restaurant  0.25
3      Sandwich Place  0.25
4         Yoga Studio  0.00


----Agincourt North,L'Amoreaux East
,Milliken,Steeles East
----
                venue  freq
0                Park  0.33
1          Playground  0.33
2    Asian Restaurant  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Albion Gardens
,Beaumond Heights,Humbergate
,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.18
1             Pharmacy  0.09
2  Fried Chicken Joint  0.09
3          Coffee Shop  0.09
4          Pizza Place  0.09


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place   0.2
1     Coffee Shop  

                        venue  freq
0               Deli / Bodega   0.2
1                        Park   0.2
2  Construction & Landscaping   0.2
3            Basketball Court   0.2
4                      Bakery   0.2


----East Birchmount Park
,Ionview,Kennedy Park----
              venue  freq
0       Coffee Shop  0.25
1        Playground  0.25
2    Discount Store  0.25
3  Department Store  0.25
4             Motel  0.00


----East Toronto----
               venue  freq
0        Coffee Shop  0.33
1               Park  0.33
2  Convenience Store  0.33
3      Movie Theater  0.00
4     Massage Studio  0.00


----Emery,Humberlea----
            venue  freq
0  Baseball Field   1.0
1     Yoga Studio   0.0
2          Lounge   0.0
3          Market   0.0
4  Massage Studio   0.0


----Fairview
,Henry Farm,Oriole
----
            venue  freq
0  Clothing Store  0.17
1     Coffee Shop  0.10
2  Cosmetics Shop  0.03
3     Candy Store  0.03
4          Bakery  0.03


----First Canadian Place,Undergroun

                 venue  freq
0            Gastropub  0.10
1          Coffee Shop  0.10
2   Italian Restaurant  0.07
3           Restaurant  0.07
4  Japanese Restaurant  0.07


----Stn A PO Boxes 25 The Esplanade
----
                venue  freq
0                Café  0.10
1      Farmers Market  0.07
2  Seafood Restaurant  0.07
3            Beer Bar  0.07
4        Cocktail Bar  0.07


----Studio District
----
                 venue  freq
0                 Café  0.13
1          Coffee Shop  0.10
2   Italian Restaurant  0.07
3  American Restaurant  0.07
4               Bakery  0.07


----The Annex,North Midtown
,Yorkville----
               venue  freq
0               Café  0.13
1     Sandwich Place  0.13
2        Coffee Shop  0.13
3        Pizza Place  0.09
4  Jewish Restaurant  0.04


----The Beaches----
                  venue  freq
0                 Trail   0.2
1  Other Great Outdoors   0.2
2     Health Food Store   0.2
3                   Pub   0.2
4           Yoga Studio   0.0


---

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide\n,King\n,Richmond\n",Steakhouse,Pizza Place,Hotel,Asian Restaurant,Café,Bar,Speakeasy,Smoke Shop,Seafood Restaurant,Plaza
1,Agincourt,Lounge,Chinese Restaurant,Breakfast Spot,Sandwich Place,Curling Ice,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
2,"Agincourt North,L'Amoreaux East\n,Milliken,Ste...",Park,Playground,Asian Restaurant,College Arts Building,College Gym,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
3,"Albion Gardens\n,Beaumond Heights,Humbergate\n...",Grocery Store,Coffee Shop,Pizza Place,Beer Store,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Liquor Store,Pharmacy,Video Store
4,"Alderwood,Long Branch",Pizza Place,Pub,Coffee Shop,Skating Rink,Gym,Pharmacy,Dance Studio,Pool,Sandwich Place,Deli / Bodega


In [91]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 2, 2, 0, 0, 0, 0, 0])

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Cupcake Shop,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Cupcake Shop,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Mexican Restaurant,Café,Breakfast Spot,Gym / Fitness Center,French Restaurant,Historic Site,Farmers Market
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Accessories Store,Event Space,Miscellaneous Shop,Fraternity House,Boutique,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gym,Park,Mexican Restaurant,Café,Creperie,Portuguese Restaurant,Burrito Place


In [117]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)



# add markers to the map
markers_colors = []
for lat, lon, neigh, pc, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(neigh) + '(' + str(pc) + '): Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)


In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0.0,Coffee Shop,Park,Bakery,Mexican Restaurant,Café,Breakfast Spot,Gym / Fitness Center,French Restaurant,Historic Site,Farmers Market
3,North York,0.0,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Accessories Store,Event Space,Miscellaneous Shop,Fraternity House,Boutique,Vietnamese Restaurant
4,Queen's Park,0.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gym,Park,Mexican Restaurant,Café,Creperie,Portuguese Restaurant,Burrito Place
6,Scarborough,0.0,Fast Food Restaurant,Print Shop,Women's Store,Cupcake Shop,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
7,North York,0.0,Baseball Field,Gym / Fitness Center,Café,Japanese Restaurant,Caribbean Restaurant,Diner,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store
8,East York,0.0,Pizza Place,Fast Food Restaurant,Breakfast Spot,Bank,Gastropub,Café,Gym / Fitness Center,Pet Store,Pharmacy,Intersection
9,Downtown Toronto,0.0,Café,Clothing Store,Ramen Restaurant,Steakhouse,Sporting Goods Shop,Spa,Movie Theater,Japanese Restaurant,Sandwich Place,Diner
13,North York,0.0,Coffee Shop,Gym,Asian Restaurant,Beer Store,Sandwich Place,Bike Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant
14,East York,0.0,Park,Cosmetics Shop,Skating Rink,Beer Store,Curling Ice,Bus Stop,Athletics & Sports,Pharmacy,Comic Shop,Dance Studio
15,Downtown Toronto,0.0,Coffee Shop,Gastropub,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Speakeasy,Latin American Restaurant,Diner,Middle Eastern Restaurant


In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,1.0,Playground,Women's Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
83,Central Toronto,1.0,Tennis Court,Playground,Creperie,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2.0,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Cupcake Shop,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
12,Scarborough,2.0,Bar,Construction & Landscaping,Women's Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
17,Etobicoke,2.0,Liquor Store,Convenience Store,Beer Store,Pharmacy,Pizza Place,Café,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
22,Scarborough,2.0,Coffee Shop,Korean Restaurant,Convenience Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
34,North York,2.0,Coffee Shop,Falafel Restaurant,Massage Studio,Bar,Caribbean Restaurant,Metro Station,Miscellaneous Shop,Dance Studio,Drugstore,Dog Run
38,Scarborough,2.0,Coffee Shop,Playground,Discount Store,Department Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Diner
50,North York,2.0,Empanada Restaurant,Pharmacy,Pizza Place,Women's Store,Cuban Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
63,York,2.0,Bus Line,Grocery Store,Pizza Place,Convenience Store,Dessert Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Women's Store
70,Etobicoke,2.0,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Intersection,Cupcake Shop,Drugstore,Dog Run,Discount Store,Diner
72,North York,2.0,Coffee Shop,Pharmacy,Pizza Place,Discount Store,Grocery Store,Dumpling Restaurant,Drugstore,Dog Run,Cuban Restaurant,Diner


In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3.0,Baseball Field,Women's Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
101,Etobicoke,3.0,Baseball Field,Women's Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant


In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Cupcake Shop,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
10,North York,4.0,Park,Pizza Place,Japanese Restaurant,Pub,Cuban Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
11,Etobicoke,4.0,Bank,Women's Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
21,York,4.0,Park,Fast Food Restaurant,Pharmacy,Market,Women's Store,Gastropub,Garden Center,General Entertainment,Discount Store,Diner
35,East York,4.0,Coffee Shop,Park,Convenience Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
40,North York,4.0,Park,Other Repair Shop,Airport,Cuban Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
45,North York,4.0,Cafeteria,Women's Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
49,North York,4.0,Park,Basketball Court,Construction & Landscaping,Bakery,Deli / Bodega,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
61,Central Toronto,4.0,Park,Bus Line,Swim School,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
64,York,4.0,Park,Women's Store,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
